In [ ]:
## Load data for eye position and a CEBRA-Time model ##

# for a single fish
filename = FILENAME
filename_trunc = filename.split('/')[-1][:-3] # fish and date only
data_folder = 'data/'

# choose where in dataset to sample
start, stop = 0, 0+TIMESTEPS


# extract eye position and neural data
# do not attempt to load the entire file 
print("Accessing data...")
with h5py.File(filename, 'r') as f:

    # eye position
    eye_pos_l = f['visuomotor']['eye_pos']['Left']
    eye_pos_r = f['visuomotor']['eye_pos']['Right']
    print(f"Full eye position dataset shape is: {eye_pos_l.shape}")

    # neural 
    neural = f['rois']['dfof']
    print(f"Full neural dataset shape is: {neural.shape}")

    # select first TIMESTEPS timesteps and random ROIS rois
    # neural
    neural_indexes = np.sort(
                        np.random.choice(
                                    np.arange(neural.shape[1]), size=ROIS, replace=False
                                    )
                        )
    neural = np.array(neural[start:stop, neural_indexes])

    # eye position
    eye_pos_l = np.array(eye_pos_l[start:stop])
    eye_pos_r = np.array(eye_pos_r[start:stop])

    print(f"Truncated dataset shapes are:\n \
            eye_pos_l: {eye_pos_l.shape}\n \
            eye_pos_r: {eye_pos_r.shape}\n \
            neural: {neural.shape}")

    assert(neural.shape == (TIMESTEPS, ROIS))


    # save datasets
    filename_eye_pos = f'{filename[-12:-3]}_eye_pos.npz'
    filename_dfof = f'{filename[-12:-3]}_dfof.npz'
    np.savez(f'{data_folder} + {filename_eye_pos}', eye_pos_l=eye_pos_l, eye_pos_r=eye_pos_r)
    np.savez(f'{data_folder} + {filename_dfof}', neural=neural)

print("Data accessed.")

eye_pos_l = cebra.load_data(f'{data_folder}{filename_eye_pos}', key="eye_pos_l")
print(f"{filename_eye_pos}_left loaded.")
eye_pos_r = cebra.load_data(filename_eye_pos, key="eye_pos_r")
print(f"{filename_eye_pos}_right loaded.")
neural = cebra.load_data(f'{data_folder}{filename_eye_pos}', key="neural")
print(f"{filename_dfof} loaded.")




In [ ]:
## truncate nueral data to be a random selection of ROIs and a consecutive sequence of timesteps, with predefined shape ##

import numpy as np

# truncate neural
# select first TIMESTEPS timesteps and random ROIS rois
neural_indexes = np.sort(
                    np.random.choice(
                                np.arange(neural.shape[1]), size=ROIS, replace=False
                                )
                    )
neural = np.array(neural[start:stop, neural_indexes])
print(f'Truncated neural dataset shape is: {neural.shape}')

In [ ]:
##  singular (non-loop) implementation of creating binary masks for stimulus 'on' frames ##

import h5py

filepath = '/media/storage/DATA/lfads_export/f1_221027.h5'
stim_types = {'left_spot':0, 'right_spot':1,  \
              'open_loop_grating':2, 'closed_loop_grating':3}
neural = np.zeros((100000,1))

with h5py.File(filepath, 'r') as f:

    # get stimulus presentations
    stimuli = f['visuomotor']['presentations']
    stim_type = stimuli['stim_type'].astype(int)
    stim_onset_fr = stimuli['onset_frame'].astype(int)
    stim_end_fr = stimuli['offset_frame'].astype(int)


    # find the presentation indexes with left or right spots
    stim_pres_idx_l = np.where(np.isin(stim_type, 1))[0]    # left spots
    stim_pres_idx_r = np.where(np.isin(stim_type, 2))[0]    # right spots

    # index stim onset frames with the presentation indexes
    stim_onset_fr = stimuli['onset_frame'].astype(int)
    stim_pres_fr_l = stim_onset_fr[stim_pres_idx_l]
    stim_pres_fr_r = stim_onset_fr[stim_pres_idx_r]

    # index stim end frames with the presentation indexes
    stim_end_fr = stimuli['offset_frame'].astype(int)
    stim_end_fr_l = stim_end_fr[stim_pres_idx_l]
    stim_end_fr_r = stim_end_fr[stim_pres_idx_r]

    # create masks of stim onset/stim end
    # left spot
    stim_on_l = np.zeros(neural.shape[0]).astype(int)
    stim_on_l[[stim_pres_fr_l, stim_end_fr_l]] = 1
    np.bitwise_xor.accumulate(stim_on_l) | stim_on_l
    # right spot
    stim_on_r = np.zeros(neural.shape[0]).astype(int)
    stim_on_r[[stim_pres_fr_r, stim_end_fr_r]] = 1
    np.bitwise_xor.accumulate(stim_on_r) | stim_on_r

    # find duration (in frames) of each presentation
    # (neural recording is at 5Hz)
    stim_dur_l = stim_end_fr_l - stim_pres_fr_l
    stim_dur_r = stim_end_fr_r - stim_pres_fr_r